In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = r'model/RTA'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8) 

In [ ]:
from datasets import load_dataset, concatenate_datasets

test_dataset = load_dataset('csv', data_files='data/dataset_5fold_1/dataset_fold_4.csv')['train']

print(f'test_dataset:{test_dataset}')

In [ ]:
def preprocess_data(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

encoded_test_dataset = test_dataset.map(preprocess_data, batched=True)

In [ ]:
# 生成测试集的元特征
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainingArguments, Trainer
import numpy as np
import pandas as pd
import torch

model = AutoModelForSequenceClassification.from_pretrained("./output/RTA-ensemble-v2-4/checkpoint-1200")

class_names = [
    'code-simplification-optimization-defects',
    'control-flow-optimization-defects',
    'data-flow-analysis-optimization-defects',
    'infrastructure-defects',
    'interprocedural-optimization-defects',
    'memory-optimization-defects',
    'numerical-analysis-optimization-defects',
    'vectorization-defects'
]

# 配置训练参数
training_args = TrainingArguments(
    output_dir='./results', 
    per_device_eval_batch_size=32,  
    do_train=False,  # 关闭训练
    do_eval=True,    # 打开评估模式
    report_to="none",
)

# 创建Trainer实例
trainer = Trainer(
    model=model,                        
    args=training_args,  
)

test_predictions, test_labels, _ = trainer.predict(encoded_test_dataset)

# 获取预测的类别
test_predicted_classes = np.argmax(test_predictions, axis=1)

# 将预测的 logits 转换为概率分布（soft labels）
test_soft_labels = torch.nn.functional.softmax(torch.tensor(test_predictions), dim=1).numpy()

# 计算分类准确度
accuracy = accuracy_score(test_labels, test_predicted_classes)
report = classification_report(test_labels, test_predicted_classes, target_names=class_names, digits=4)
print(f"Accuracy: {accuracy:.4f}")
print("分类报告：")
print(report)

# # 保存元特征
# column_names = [f"rta-{i}" for i in range(8)]
# test_predictions_df = pd.DataFrame(test_soft_labels, columns=column_names)
# test_predictions_df.to_csv("data/soft-label/rta-soft-labels-v2-4.csv", index=False)
# print("\n预测结果已保存")

In [ ]:
encoded_test_dataset[0]

In [ ]:
test_predictions.shape